In [36]:
# onedrive = r"C:\Users\Mike.Kryajak\OneDrive"
onedrive = r"C:\Users\mikek\OneDrive"
# %matplotlib ipympl
# %matplotlib qt
# %matplotlib widget
%matplotlib inline

from boututils.datafile import DataFile
from boutdata.collect import collect
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys, pathlib
import platform
import traceback
import xarray as xr
import xbout
import scipy
import re
import netCDF4 as nc

onedrive_path = onedrive_path = str(os.getcwd()).split("OneDrive")[0] + "OneDrive"
sys.path.append(os.path.join(onedrive_path, r"Project\python-packages\sdtools"))
sys.path.append(os.path.join(onedrive_path, r"Project\python-packages\soledge"))
sys.path.append(os.path.join(onedrive_path, r"Project\python-packages"))


# from gridtools.hypnotoad_tools import *
from gridtools.b2_tools import *
from gridtools.utils import *

from hermes3.case_db import *
from hermes3.load import *
from hermes3.named_selections import *
from hermes3.plotting import *
from hermes3.grid_fields import *
from hermes3.accessors import *
from hermes3.utils import *
from code_comparison.code_comparison import *

import gridtools.solps_python_scripts.setup
from gridtools.solps_python_scripts.plot_solps       import plot_1d, plot_2d, plot_wall_loads
from gridtools.solps_python_scripts.read_ft44 import read_ft44

%load_ext autoreload
%autoreload 2


print("Done")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Done


In [4]:
path = os.path.join(onedrive_path, r"Project\collab\tech\compare_data\HERMES_SOLEDGE.S33600_Tamb_case.SEE=0.BGK=OFF.SOLEDGE.reactions.input.dat.OK.Js=OFF.GAMMAE=4.5.eirene_ionising_core=0.BCCON=16")
(neut, wld) = read_ft44(where = path, save = True)

fort.44 loaded from .pkl


In [13]:
wld.keys()

# wldnek: heat transferred by neutrals (W)
# 

dict_keys(['wldnek', 'wldnep', 'wldna', 'ewlda', 'wldnm', 'ewldm', 'wldra', 'wldrm', 'poly', 'wldpp', 'wldpa', 'wldpm', 'wldpeb', 'wldspt', 'wldspta', 'wldsptm', 'isrftype', 'wlarea', 'wlabsrp', 'wlpump', 'eirdiag_nds_ind', 'eirdiag_nds_typ', 'eirdiag_nds_srf', 'eirdiag_nds_start', 'eirdiag_nds_end', 'sarea_res', 'wldna_res', 'wldnm_res', 'ewlda_res', 'ewldm_res', 'ewldea_res', 'ewldem_res', 'ewldrp_res', 'ewldmr_res', 'wldspt_res', 'wldspta_res', 'wldsptm_res', 'wlpump_res', 'ewldt_res'])

In [23]:
[print(f"{x}: {wld[x].shape}") for x in ["wldnek", "poly", "wlarea"]]

wldnek: (55, 15)
poly: (4, 42)
wlarea: (55,)


[None, None, None]

In [12]:
%matplotlib widget

- Looks like there are 55 elements but only 42 are in poly, which is why Matteo only plots these.
- wldnek is the total energy deposited by the neutrals on the target in W
- wldnep is the total potential energy of the neutrals in W
- wlarea is the area but it's not needed since the above are already in W

In [12]:
neutrals_path

'c:\\collab\\tech\\compare_data\\HERMES_SOLEDGE.S33600_Tamb_case.SEE=0.BGK=OFF.SOLEDGE.reactions.input.dat.OK.Js=OFF.GAMMAE=4.5.eirene_ionising_core=0.BCCON=16\\b2.neutrals.parameters'

In [34]:
# - Strata are W, E, W, E, S, S, S, N, N, C, C, V, T
# - N, S, W, E are SOL, core/PFR, inner lower target and outer lower target respectively
# - V is volume recombination source
# - C is gas puff source
# - T is time dependent source
# rcpos is same as BCPOS: for N, S, X it specifies row index. For W, E, Y it specifies column index.
# rcstart is the start coordinate of the strata
# rcend is the end coordionate of the strata

neutrals_path = os.path.join(onedrive_path, r"Project\collab\tech\compare_data\HERMES_SOLEDGE.S33600_Tamb_case.SEE=0.BGK=OFF.SOLEDGE.reactions.input.dat.OK.Js=OFF.GAMMAE=4.5.eirene_ionising_core=0.BCCON=16\b2.neutrals.parameters")
strata_settings = pd.read_csv(neutrals_path, names = [x for x in range(17)], sep = ",|=")
strata_settings = strata_settings.set_index(0, drop =True)
strata_settings = strata_settings.transpose()
strata_settings = strata_settings[["crcstra", "rcpos", "rcstart", "rcend"]]

strata_settings

C:\Users\Mike.Kryajak\AppData\Local\Temp\ipykernel_12748\2903344782.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  strata_settings = pd.read_csv(neutrals_path, names = [x for x in range(17)], sep = ",|=")


,crcstra,rcpos,rcstart,rcend
1,'W',-1,0,35
2,'E',48,0,35
3,'W',49,0,35
4,'E',110,0,35
5,'S',-1,0,11
6,'S',-1,36,47
7,'S',-1,50,67
8,'S',-1,92,109
9,'N',36,0,47
10,'N',36,50,109


In [49]:
d = nc.Dataset(os.path.join(path, "balance.nc"))
crx = d["crx"][:]
cry = d["cry"][:]
d.close()


In [ ]:
strata = pd.DataFrame()

strata["code"] = ["W", "E", "W", "E", "S", "S", "S", "S", "N", "N", "C", "C", "V", "T"]
strata["pos"] = [-1, 48, 49, 110, -1, -1, -1, -1, 36, 36, 0, 0, 0, 0]
strata["start"] = [0, 0, 0, 0, 0, 36, 50, 92, 0, 50, 0, 0, 0, 0]
strata["end"] = [35, 35, 35, 35, 11, 47, 67, 109, 47, 109, 0, 0, 0, 0]

strata

In [43]:
wlddata = wld["wldnek"]

for i in range(wlddata.shape[1]):
    flux = sum(wlddata[:,i]) * 1e-6
    print(f"Stratum {i}: {flux:.3f} MW")
    
print(f"Sum of all strata: {np.sum(wlddata)*1e-6:.3e} MW")

Stratum 0: 1.022 MW
Stratum 1: 0.088 MW
Stratum 2: 0.090 MW
Stratum 3: 0.386 MW
Stratum 4: 0.383 MW
Stratum 5: 0.000 MW
Stratum 6: 0.000 MW
Stratum 7: 0.000 MW
Stratum 8: 0.000 MW
Stratum 9: 0.000 MW
Stratum 10: 0.003 MW
Stratum 11: 0.036 MW
Stratum 12: 0.035 MW
Stratum 13: 0.000 MW
Stratum 14: 0.000 MW
Sum of all strata: 2.044e+00 MW
